In [3]:
import tensorflow as tf
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import pickle
import time

ModuleNotFoundError: No module named 'tensorflow'

In [5]:

from xgboost import XGBClassifier
from xgboost import XGBRegressor

## Data Pedestrian

In [3]:
train = np.loadtxt('/work/users/d/d/ddinh/aaco/input_data/MelbournePedestrian/MelbournePedestrian_TRAIN.txt')
Xtrn = train[:, 1:]
Ytrn = train[:, 0]
valid = np.loadtxt('/work/users/d/d/ddinh/aaco/input_data/MelbournePedestrian/MelbournePedestrian_TEST.txt')
Xval = valid[:, 1:]
Yval = valid[:, 0]

Ytrn = np.eye(11)[Ytrn.astype(int)]
Yval = np.eye(11)[Yval.astype(int)]
Ytrn = Ytrn[:, 1:]
Yval = Yval[:, 1:]

num_ts = Xtrn.shape[1]

In [30]:
Xtrn[:2]

array([[  97.,   42.,   20.,   10.,   14.,   33.,  113.,  422.,  875.,
        1003., 1351., 1613., 2937., 2954., 2167., 2330., 2191., 2621.,
        2400., 1892., 1253.,  844.,  438.,  204.],
       [ 107.,   50.,   41.,   17.,   48.,   35.,  102.,  326.,  856.,
         956., 1397., 2079., 3570., 3857., 2999., 3032., 2794., 3061.,
        2323., 1463.,  773.,  646.,  305.,  158.]])

In [28]:
np.expand_dims(Xtrn, axis=2).shape

(1194, 24, 1)

In [26]:
train.shape

(1194, 25)

In [9]:
masksper = 512
d = Xtrn.shape[1]
X_class = np.concatenate([Xtrn]*masksper, 0)
Y_class = np.concatenate([Ytrn]*masksper, 0)
B = np.concatenate(
[np.sum(np.random.permutation(np.eye(d))[:, :np.random.randint(d)], 1, keepdims=True) for _ in range(X_class.shape[0])],
1)
B = np.float32(B.T)
X_class = np.concatenate((X_class*B, B), 1)
# Train classifier
est = XGBClassifier(n_estimators=256, device='gpu')
est.fit(X_class, Y_class)

/work/users/d/d/ddinh/.venv/lib64/python3.11/site-packages/xgboost/core.py:158: UserWarning: [14:12:08] WARNING: /workspace/src/context.cc:43: No visible GPU is found, setting device to CPU.
  warnings.warn(smsg, UserWarning)
/work/users/d/d/ddinh/.venv/lib64/python3.11/site-packages/xgboost/core.py:158: UserWarning: [14:12:08] WARNING: /workspace/src/context.cc:196: XGBoost is not compiled with CUDA support.
  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device='gpu', early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=256, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [23]:
# save model to path 
path = '/work/users/d/d/ddinh/aaco/models/'
est.save_model(path + 'pedestrian.model')

/work/users/d/d/ddinh/.venv/lib64/python3.11/site-packages/xgboost/core.py:158: UserWarning: [14:16:27] WARNING: /workspace/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)


In [16]:
Bval = np.concatenate(
  [np.sum(np.random.permutation(np.eye(d))[:, :np.random.randint(8)], 1, keepdims=True) for _ in range(Xval.shape[0])],
  1)
Bval = np.float32(Bval.T)
Xvalmasked = np.concatenate((Xval*Bval, Bval), 1)

In [12]:
val_preds = est.predict_proba(np.concatenate((Xval, np.ones_like(Xval)), 1))

In [13]:
print(np.mean(np.round(val_preds)==Yval))

0.9863058630586305


In [17]:
val_preds = est.predict_proba(Xvalmasked) 

In [18]:
print(np.mean(np.round(val_preds)==Yval))

0.9403034030340304


# DATA Digits

In [3]:

from tslearn.datasets import UCR_UEA_datasets
data_loader = UCR_UEA_datasets()

Xtrn, Ytrn, Xval, Yval = data_loader.load_dataset(
        "PenDigits")
Ytrn = np.eye(10)[Ytrn.astype(int)]
Yval = np.eye(10)[Yval.astype(int)]

In [4]:
Xtrn.shape, Xval.shape

((7494, 8, 2), (3498, 8, 2))

In [ ]:
np.transpose([X_class[0]], (0,2,1)).reshape(-1, X_class.shape[1] * X_class.shape[2])

array([[ 47.,  27.,  57.,  26.,   0.,  56., 100.,  40., 100.,  81.,  37.,
          0.,  23.,  53.,  90.,  98.]])

In [7]:
Xtrn.shape

(7494, 8, 2)

In [5]:
masksper = 256
d = Xtrn.shape[1]
X_class = np.concatenate([Xtrn]*masksper, 0)
Y_class = np.concatenate([Ytrn]*masksper, 0)
B = np.concatenate(
[np.sum(np.random.permutation(np.eye(d))[:, :np.random.randint(d)], 1, keepdims=True) for _ in range(X_class.shape[0])],
1)
B = np.float32(B.T)
B = np.expand_dims(B, axis=-1)

B = np.concatenate([B]*2, 2)

B = np.transpose(B, (0,2,1)).reshape(-1, B.shape[1] * B.shape[2])
X_class = np.transpose(X_class, (0,2,1)).reshape(-1, X_class.shape[1] * X_class.shape[2])

X_class = np.concatenate((X_class*B, B), 1)
# X_class = X_class*B
# Train classifier
est = XGBClassifier(n_estimators=256, device='gpu')
est.fit(X_class, Y_class)

/work/users/d/d/ddinh/.venv/lib64/python3.11/site-packages/xgboost/core.py:158: UserWarning: [16:12:54] WARNING: /workspace/src/context.cc:43: No visible GPU is found, setting device to CPU.
  warnings.warn(smsg, UserWarning)
/work/users/d/d/ddinh/.venv/lib64/python3.11/site-packages/xgboost/core.py:158: UserWarning: [16:12:54] WARNING: /workspace/src/context.cc:196: XGBoost is not compiled with CUDA support.
  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device='gpu', early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=256, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [6]:
# save model to path 
path = '/work/users/d/d/ddinh/aaco/models/'
est.save_model(path + 'digits.model')

/work/users/d/d/ddinh/.venv/lib64/python3.11/site-packages/xgboost/core.py:158: UserWarning: [16:18:29] WARNING: /workspace/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)


In [ ]:
B.shape

(1918464, 16)

In [ ]:
Bval = np.concatenate(
  [np.sum(np.random.permutation(np.eye(d))[:, :np.random.randint(8)], 1, keepdims=True) for _ in range(Xval.shape[0])],
  1)
Bval = np.float32(Bval.T)
Bval = np.expand_dims(Bval, axis=-1)
Bval = np.concatenate([Bval]*2, 2)
Bval = np.transpose(Bval, (0,2,1)).reshape(-1, Bval.shape[1] * Bval.shape[2])
Xval = np.transpose(Xval, (0,2,1)).reshape(-1, Xval.shape[1] * Xval.shape[2])

Xvalmasked = np.concatenate((Xval*Bval, Bval), 1)
# Xvalmasked = Xval*Bval

In [57]:
val_preds = est.predict_proba(Xvalmasked) 
print(np.mean(np.round(val_preds)==Yval))

0.9571755288736421


In [ ]:
0.9596054888507719

# Synthetic

In [9]:
import numpy as np
np.random.seed(42)

def draw_sequence(length, digit_low, digit_high, counter_low, counter_high):
    digits, counter, important_digits = [], [], []

    while len(digits) < length:
        max_count = np.random.randint(counter_low + 1, counter_high + 1)
        counter_i = np.arange(max_count, counter_low - 1, -1)
        counter.extend(counter_i)

        digits_i = np.random.randint(digit_low, digit_high + 1, len(counter_i))
        if len(digits_i) + len(digits) <= length:
            important_digits.append(digits_i[-1])
        digits.extend(digits_i)

    digits = digits[:length]
    counter = counter[:length]
    label = sum(important_digits)

    return digits, counter, label

sequence_length = 10
digit_low = 0
digit_high = 2
counter_low = 0
counter_high = 2

draw_sequence(sequence_length, digit_low, digit_high, counter_low, counter_high)
train_size = 50000
test_size = 10000

def generate_dataset(size):
    digits_list = []
    counter_list = []
    labels = []
    for _ in range(size):
        digits, counter, label = draw_sequence(sequence_length, digit_low, digit_high, counter_low, counter_high)
        digits_list.append(digits)
        counter_list.append(counter)
        labels.append(label)
    return np.array(digits_list), np.array(counter_list), np.array(labels)

X_train_digits, X_train_counter, y_train = generate_dataset(train_size)
X_test_digits, X_test_counter, y_test = generate_dataset(test_size)

np.save('/work/users/d/d/ddinh/aaco/input_data/Synthetic_raw/train_digits.npy', X_train_digits)
np.save('/work/users/d/d/ddinh/aaco/input_data/Synthetic_raw/train_counter.npy', X_train_counter)
np.save('/work/users/d/d/ddinh/aaco/input_data/Synthetic_raw/train_labels.npy', y_train)
np.save('/work/users/d/d/ddinh/aaco/input_data/Synthetic_raw/test_digits.npy', X_test_digits)
np.save('/work/users/d/d/ddinh/aaco/input_data/Synthetic_raw/test_counter.npy', X_test_counter)
np.save('/work/users/d/d/ddinh/aaco/input_data/Synthetic_raw/test_labels.npy', y_test)

In [6]:
import numpy as np

In [7]:
X_train_digits = np.load('/work/users/d/d/ddinh/aaco/input_data/Synthetic_raw/train_digits.npy')
X_train_counter = np.load('/work/users/d/d/ddinh/aaco/input_data/Synthetic_raw/train_counter.npy')
y_train = np.load('/work/users/d/d/ddinh/aaco/input_data/Synthetic_raw/train_labels.npy')
X_test_digits = np.load('/work/users/d/d/ddinh/aaco/input_data/Synthetic_raw/test_digits.npy')
X_test_counter = np.load('/work/users/d/d/ddinh/aaco/input_data/Synthetic_raw/test_counter.npy')
y_test = np.load('/work/users/d/d/ddinh/aaco/input_data/Synthetic_raw/test_labels.npy')

In [8]:
y_train = np.eye(np.max(y_train) + 1)[y_train]
y_test = np.eye(np.max(y_test) + 1)[y_test]

In [9]:
X_train = np.concatenate([X_train_digits, X_train_counter], axis=1)
X_test = np.concatenate([X_test_digits, X_test_counter], axis=1)

In [ ]:
# xgb 
masksper = 512
d = X_train.shape[1]
X_class = np.concatenate([X_train]*masksper, 0)
Y_class = np.concatenate([y_train]*masksper, 0)
B = np.concatenate(
[np.sum(np.random.permutation(np.eye(d))[:, :np.random.randint(d)], 1, keepdims=True) for _ in range(X_class.shape[0])],
1)
B = np.float32(B.T)
X_class = np.concatenate((X_class*B, B), 1)

In [ ]:
# Train classifier
from xgboost import XGBClassifier
est = XGBClassifier(n_estimators=256, device='gpu')
est.fit(X_class, Y_class)

In [ ]:
# save model to path
path = '/work/users/d/d/ddinh/aaco/models/'
est.save_model(path + 'synthetic_raw.model')

In [ ]:
Bval = np.concatenate(
  [np.sum(np.random.permutation(np.eye(d))[:, :np.random.randint(8)], 1, keepdims=True) for _ in range(X_test.shape[0])],
  1)
Bval = np.float32(Bval.T)
Xvalmasked = np.concatenate((X_test*Bval, Bval), 1)

val_preds = est.predict_proba(Xvalmasked)

print(np.mean(np.round(val_preds)==y_test))

# mean number of masks Bval
print(np.mean(np.sum(Bval, axis=1)))